
# Snippets for different ways to call LLMs

In [ ]:
#LLM Config

from openai import OpenAI
import os
os.environ["OPENAI_API_KEY"] = ""  # Replace with your actual key
client = OpenAI()


## Simple Prompting

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4o",
    input="What is the capital of France?"
)

print(response.output_text)

The capital of France is Paris.


## Instruction Following

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4.1",
    instructions="Talk like a pirate.",
    input="What is the capital of France?",
)

print(response.output_text)

YARRR! The capital o' France be Paris, matey! A fine city fer treasure huntin' an’ enjoyin’ a mug o’ rum by the River Seine. Hoist yer sails an’ set a course fer Paris, ye landlubber! Arrr!


## Adding roles in Messages

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4.1",
    input=[
        {
            "role": "developer",
            "content": "Talk like a pirate."
        },
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ]
)

print(response.output_text)

Arrr! The grand capital o’ France be Paris, matey! ‘Tis a city of lights, fine grub, and treasures aplenty! If ye be hankerin’ for the Eiffel Tower or fancy a spot o’ plunder at the Louvre, Paris be the place to set yer course!


## Getting Structured Outputs from LLMs

In [ ]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

response = client.responses.parse(
    model="gpt-4o",
    input=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Bruce and Arthur are going to a boxing match on Saturday.",
        },
    ],
    text_format=CalendarEvent,
)

event = response.output_parsed
print(event)

name='Boxing Match' date='Saturday' participants=['Bruce', 'Arthur']


## Structured Outputs for chain-of-thought math tutoring

In [ ]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

response = client.responses.parse(
    model="gpt-4o",
    input=[
        {
            "role": "system",
            "content": "You are a helpful math tutor. Guide the user through the solution step by step.",
        },
        {"role": "user", "content": "how can I solve 8x + 7 = -23"},
    ],
    text_format=MathReasoning,
)

math_reasoning = response.output_parsed
print(math_reasoning)

steps=[Step(explanation='We need to solve the equation for \\( x \\). Start by isolating the term with \\( x \\) on one side of the equation.', output='Subtract 7 from both sides of the equation to get rid of the constant term on the left side: \n\n\\[ 8x + 7 - 7 = -23 - 7 \\]'), Step(explanation='Simplify both sides by performing the subtraction.', output='This results in: \n\n\\[ 8x = -30 \\]'), Step(explanation='Now, isolate \\( x \\) by dividing both sides of the equation by 8.', output='Divide both sides by 8: \n\n\\[ x = \\frac{-30}{8} \\]'), Step(explanation='Simplify the fraction by dividing the numerator and denominator by their greatest common divisor, which is 2.', output='Simplify the fraction: \n\n\\[ x = \\frac{-30 \\div 2}{8 \\div 2} = \\frac{-15}{4} \\]')] final_answer='x = \\frac{-15}{4}'


##  Function Calling - Enable models to fetch data and take actions.

In [ ]:
from openai import OpenAI

client = OpenAI()

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]

response = client.responses.create(
    model="gpt-4o",
    input=[{"role": "user", "content": "What is the weather in Paris?"}],
    tools=tools
)

print(response.output)

[ResponseFunctionToolCall(arguments='{"location":"Paris, France"}', call_id='call_QZhR5dDSqJBLTMiQavDsthY9', name='get_weather', type='function_call', id='fc_686695bdf16881998ff357d3be1c6a7b01d7db05f283da0b', status='completed')]


## Providing Multiple Tools

In [ ]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
},{
    "type": "function",
    "name": "send_email",
    "description": "Send an email to a given recipient with a subject and message.",
    "parameters": {
        "type": "object",
        "properties": {
            "to": {
                "type": "string",
                "description": "The recipient email address."
            },
            "subject": {
                "type": "string",
                "description": "Email subject line."
            },
            "body": {
                "type": "string",
                "description": "Body of the email message."
            }
        },
        "required": [
            "to",
            "subject",
            "body"
        ],
        "additionalProperties": False
    }
}]

response = client.responses.create(
    model="gpt-4o",
    input=[{"role": "user", "content": "What is the capital of France?"}],
    tools=tools
)

print(response.output)


response = client.responses.create(
    model="gpt-4o",
    input=[{"role": "user", "content": "What is the current weather in Paris?"}],
    tools=tools
)

print(response.output)


response = client.responses.create(
    model="gpt-4o",
    input=[{"role": "user", "content": "Can you send an email to bruce@wayne.co and alfred@wayne.co asking if the batmobile is ready?"}],
    tools=tools
)

print(response.output)


[ResponseOutputMessage(id='msg_686697b64efc819bb38c48aa0641c08e039ddffeffcb0ff2', content=[ResponseOutputText(annotations=[], text='The capital of France is Paris.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]
[ResponseFunctionToolCall(arguments='{"location":"Paris, France"}', call_id='call_ZocN2cePHnpNc4J0DS4oROP6', name='get_weather', type='function_call', id='fc_686697b72f90819a9a34fa87212863660dd0c8a436b5e806', status='completed')]
[ResponseFunctionToolCall(arguments='{"to":"bruce@wayne.co","subject":"Is the Batmobile Ready?","body":"Hi Bruce,\\n\\nI wanted to check if the Batmobile is ready.\\n\\nThanks!"}', call_id='call_KYfj2S4gpyE62QlwbXrjRzSU', name='send_email', type='function_call', id='fc_686697b83c40819aaf9a649faaa142cb04c44fc25c5f07cf', status='completed'), ResponseFunctionToolCall(arguments='{"to":"alfred@wayne.co","subject":"Is the Batmobile Ready?","body":"Hi Alfred,\\n\\nI wanted to check if the Batmobile is ready.\\n\\nTh